In [187]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of data failed: Traceback (most recent call last):
  File "/home/tormod/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/tormod/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/tormod/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/tormod/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/tormod/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/tormod/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/tormod/.l

In [188]:
import os
import numpy as np
import mmcv
import matplotlib
import matplotlib.pyplot as plt
import data
import joblib
import torch
import bbox
import utils
from tqdm.auto import tqdm, trange
from bbox import cxcywh2cs
#from mesh import render_vids
from meva.lib.vibe_loss import VIBELoss
from meva.utils.video_config import update_cfg
from meva.utils import image_utils, kp_utils
from meva.utils.renderer import Renderer
from meva.utils.demo_utils import convert_crop_cam_to_orig_img

In [189]:
cd = data.ClimbingDataset('test', preload_all=True)
first = cd.get(0)
first.keys()

/home/tormod/Documents/APEC/meva/utils/image_utils.py:384: RuntimeWarning: invalid value encountered in double_scalars
  w = h = np.where(w / h > 1, w, h)


1521

In [75]:
res = joblib.load('/media/tormod/Den Lille/Thesis/meva_results/meva_output.pkl')

In [4]:
seqidx = 0
frameid = 6
seqinfo = cd.get(seqidx)
inv_trans = seqinfo['inv_trans'][0]
kp_2d = res['joints2d'][seqidx, frameid]
kp_2d = image_utils.normalize_2d_kp(kp_2d, inv=True)
kp_2d = image_utils.trans_points2d(kp_2d, inv_trans)
plt.imshow(seqinfo['raw_imgs'][frameid])
plt.scatter(kp_2d[:,0], kp_2d[:,1])
plt.show();


NameError: name 'res' is not defined

In [ ]:
render_vids(cd, res, '/media/tormod/Den Lille/Thesis/meva_out')

In [ ]:
config_file = f"meva/cfg/train_meva_2.yml"
cfg = update_cfg(config_file)
loss = VIBELoss(
    e_loss_weight=cfg.LOSS.KP_2D_W,
    e_3d_loss_weight=cfg.LOSS.KP_3D_W,
    e_pose_loss_weight=cfg.LOSS.POSE_W,
    e_shape_loss_weight=cfg.LOSS.SHAPE_W,
)

In [ ]:
def vibe_loss(seqidx, res, cd):
    pred_seq = res['joints2d'][seqidx]
    pred_seq = image_utils.normalize_2d_kp(pred_seq, inv=True)
    pred_seq = torch.Tensor(pred_seq)

    seq_info = cd.get(seqidx)
    gt_seq = torch.Tensor(seq_info['kp_2d'])
    return loss.keypoint_loss(pred_seq, gt_seq, 1,1)
    
vibe_loss(0, res, cd)

In [184]:
def compute_oks(seqidx, res, cd):
    seq_info = cd.get(seqidx)
    inv_trans = seq_info['inv_trans']
    print(inv_trans.shape)
    scales = seq_info['bboxes'][:,-1]
    pred_seq = res['joints2d'][seqidx].copy()
    gt_seq = seq_info['kp_2d'].copy()
    return utils.oks(pred_seq, gt_seq, scales, inv_trans)
    

In [185]:
oks_res = compute_oks(0, res, cd)
oks_res.mean()

(90, 2, 3)


0.7996725648961384

In [19]:
oks_res = compute_oks(0, res, cd)
oks_res.mean()

0.9115043336011293

In [ ]:
# TODO compute AP AR https://cocodataset.org/#keypoints-eval